<a href="https://colab.research.google.com/github/sam505/Machine_Learning/blob/master/PYTORCH_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torchvision
import torch
import torch.nn as nn
from torch import optim
import torch.utils.data
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([transforms.Resize((28,28)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])

train_batch_size = 32
test_batch_size = 64

mnist_train = torchvision.datasets.MNIST(root="/", download=True, transform=transform, train=True)
mnist_test = torchvision.datasets.MNIST(root="/", download=True, transform=transform, train=False)

train_loader = torch.utils.data.DataLoader(mnist_train,
                                          shuffle=True,
                                          batch_size=train_batch_size,
                                          num_workers=2)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          shuffle=False,
                                          batch_size=test_batch_size,
                                          num_workers=2)

In [3]:
len(mnist_train[0])

2

In [4]:
len(mnist_test)

10000

In [5]:
writer = SummaryWriter()


In [6]:
class LeNet(nn.Module):
	def __init__(self):
		super(LeNet, self).__init__()
		self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
		self.conv2 = nn.Conv2d(6, 16, (5,5))
		self.fc1   = nn.Linear(16*5*5, 120)
		self.fc2   = nn.Linear(120, 84)
		self.fc3   = nn.Linear(84, 10)
	def forward(self, x):
		x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
		x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
		x = x.view(-1, self.num_flat_features(x))
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		return x
	def num_flat_features(self, x):
		size = x.size()[1:]
		num_features = 1
		for s in size:
			num_features *= s
		return num_features

In [7]:
def train(model, optimizer, epoch, train_loader, log_interval):
    # State that you are training the model
    model.train()

    # define loss function
    loss_fn = nn.CrossEntropyLoss()

    # Iterate over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Wrap the input and target output in the `Variable` wrapper
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
          data = data.to("cuda")
          target = target.to("cuda")

        # Clear the gradients, since PyTorch accumulates them
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        # Backward propagation
        loss.backward()

        # Update the parameters(weight,bias)
        optimizer.step()

        # print log
        if batch_idx % log_interval == 0:
            
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.data.item()))

In [8]:
def test(model, epoch, test_loader):
    # State that you are testing the model; this prevents layers e.g. Dropout to take effect
    model.eval()

    # Init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    # define loss function
    loss_fn = nn.CrossEntropyLoss(size_average=False)

    # Iterate over data
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
          data = data.to("cuda")
          target = target.to("cuda")

        # Forward propagation
        output = model(data)

        # Calculate & accumulate loss
        test_loss += loss_fn(output, target).data.item()

        # Get the index of the max log-probability (the predicted output label)
        pred = np.argmax(output.cpu().data, axis=1)

        # If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.cpu().data).sum()

    # Print log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
# Provide seed for the pseudorandom number generator s.t. the same results can be reproduced
torch.manual_seed(123)

model = LeNet()

check_gpu = torch.cuda.is_available()
if check_gpu:
	model = model.cuda()
	print ('USING GPU')
else:
	print ('USING CPU')

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
log_interval = 100
for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

USING GPU
Train set, Epoch 1 [0/60000 (0%)]	Loss: 2.293154
Train set, Epoch 1 [3200/60000 (5%)]	Loss: 0.704558
Train set, Epoch 1 [6400/60000 (11%)]	Loss: 0.203603
Train set, Epoch 1 [9600/60000 (16%)]	Loss: 0.209678
Train set, Epoch 1 [12800/60000 (21%)]	Loss: 0.066283
Train set, Epoch 1 [16000/60000 (27%)]	Loss: 0.086467
Train set, Epoch 1 [19200/60000 (32%)]	Loss: 0.099034
Train set, Epoch 1 [22400/60000 (37%)]	Loss: 0.051752
Train set, Epoch 1 [25600/60000 (43%)]	Loss: 0.085965
Train set, Epoch 1 [28800/60000 (48%)]	Loss: 0.065832
Train set, Epoch 1 [32000/60000 (53%)]	Loss: 0.010793
Train set, Epoch 1 [35200/60000 (59%)]	Loss: 0.089628
Train set, Epoch 1 [38400/60000 (64%)]	Loss: 0.025988
Train set, Epoch 1 [41600/60000 (69%)]	Loss: 0.052667
Train set, Epoch 1 [44800/60000 (75%)]	Loss: 0.038602
Train set, Epoch 1 [48000/60000 (80%)]	Loss: 0.198402
Train set, Epoch 1 [51200/60000 (85%)]	Loss: 0.009598
Train set, Epoch 1 [54400/60000 (91%)]	Loss: 0.020831
Train set, Epoch 1 [57600/6

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 0.0527, Accuracy: 9839/10000 (98.39%)

Train set, Epoch 2 [0/60000 (0%)]	Loss: 0.041017
Train set, Epoch 2 [3200/60000 (5%)]	Loss: 0.031820
Train set, Epoch 2 [6400/60000 (11%)]	Loss: 0.084900
Train set, Epoch 2 [9600/60000 (16%)]	Loss: 0.009972
Train set, Epoch 2 [12800/60000 (21%)]	Loss: 0.014139
Train set, Epoch 2 [16000/60000 (27%)]	Loss: 0.012752
Train set, Epoch 2 [19200/60000 (32%)]	Loss: 0.072398
Train set, Epoch 2 [22400/60000 (37%)]	Loss: 0.038778
Train set, Epoch 2 [25600/60000 (43%)]	Loss: 0.067235
Train set, Epoch 2 [28800/60000 (48%)]	Loss: 0.072021
Train set, Epoch 2 [32000/60000 (53%)]	Loss: 0.008805
Train set, Epoch 2 [35200/60000 (59%)]	Loss: 0.006772
Train set, Epoch 2 [38400/60000 (64%)]	Loss: 0.198718
Train set, Epoch 2 [41600/60000 (69%)]	Loss: 0.010310
Train set, Epoch 2 [44800/60000 (75%)]	Loss: 0.146984
Train set, Epoch 2 [48000/60000 (80%)]	Loss: 0.264490
Train set, Epoch 2 [51200/60000 (85%)]	Loss: 0.113121
Train set, Epoch 

In [10]:
len(mnist_train)

60000

2021-06-14 17:41:32.410957: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.5.0 at http://744932252c76:6006/ (Press CTRL+C to quit)
^C
